<a href="https://colab.research.google.com/github/MinsooKwak/Kaggle_note/blob/main/Detection/FathomNet_2023/FathomNet_2023_baseline_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필사 참고: https://www.kaggle.com/code/ted0071/fathomnet-2023-baseline-yolov8

### Data 가져오기

In [1]:
from google.colab import files

files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
# Download Dataset
!kaggle competitions download -c fathomnet-out-of-sample-detection

 77% 1.00M/1.30M [00:00<00:00, 2.03MB/s]
100% 1.30M/1.30M [00:00<00:00, 2.49MB/s]


In [3]:
!unzip -qq "/content/fathomnet-out-of-sample-detection.zip"

In [4]:
ls

category_key.csv                       multilabel_classification/
demo_download.ipynb                    object_detection/
download_images.py                     requirements.txt
fathomnet-out-of-sample-detection.zip  sample_data/
kaggle.json                            sample_submission.csv


### 라이브러리 / 모듈 import

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# ultralytics library 설치 (컴퓨터 비전, 딥러닝 작업 지원 패키지)
# '& / dev/null' : 로그 및 출력 메세지 표시하지 않게 함.
'''리눅스/ 유닉스 환경에서 사용되는 리다이렉션 기호
    로그 및 출력 메세지를 "/dev/null" 이라는 특수 파일로 리다이렉트하라
    데이터 버리는 용도로 사용 (출력 보이지 않도록 하고, 설치 과정 로그 메세지 무시 가능)'''

!pip install ultralytics &> /dev/null

In [ ]:
import os
import ast    # 문법 구조화 모듈 # 딕셔너리 형태 문자열을 딕셔너리로 변경
import shutil # 파일, 디렉토리에 대한 명령 모듈

import numpy as np
import pandas as pd
import json
from PIL import Image, ImageDraw, ImgaeFont # 반투명 도형 그리기 / 글자 쓰기

import urllib.request, io
from tqdm.notebook import tqdm
from IPython.display import Image # 사진 호출
from pathlib import Path  # 파일 경로를 객체로 다루자

- shutil 모듈 설명

  ```
  1) rmtree(path, ignore_errors=False, onerror=None) : 해당 디렉토리 전체 삭제
  2)copyfile(src, dst, *, follow_symlinks=True) : 복사
    # src : 복사시킬 파일 경로
    # dst : 복사된 파일이 저장될 경로
  3) copytree(src, dst, ...) : src를 루트로 하는 전체 디렉토리 dst에 재귀적 복사
  4) move(src, dst) : src 루트로 하는 전체 디렉토리 트릴르 dst 디렉토리에 재귀적 이동
  ```



- urllib 라이브러리 설명 : https://han-py.tistory.com/320
- tqdm :

  https://velog.io/@hyunicecream/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%9C%A0%EC%9A%A9%ED%95%9C-%EA%B8%B0%EB%8A%A5-tqdm-and-tqdmnotebook
- pathlib : https://brownbears.tistory.com/415

### torch 버전 확인

In [9]:
import torch
print(torch.__version__)

2.0.1+cu118


### Import Model : YOLO v8

- https://github.com/ultralytics/ultralytics
- Task : Detection, Segmentation, Classification, Pose

In [10]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/107.7 GB disk)


In [15]:
from fastai.imports import *

In [16]:
from pandas.io.sql import DatabaseError
class Cfg:
  # 루트 경로
  DATA_EXPLORING_ROOT = Path('/content')
  INPUT_ROOT = Path('/content/object_detection')
  OUTPUT_ROOT = Path('/content/working/')

  # 이미지 경로
  IMAGES_ROOT = DATA_EXPLORING_ROOT / 'images'  # 이미지 데이터의 루트 경로
  TRAIN_IMAGES_ROOT = IMAGES_ROOT / 'train'
  EVAL_IMAGES_ROOT = IMAGES_ROOT / 'eval'
  DATASET_ROOT = OUTPUT_ROOT / 'dataset'

  # 파일 경로
  TRAIN_IMAGE_DATA = DATA_EXPLORING_ROOT / 'train_image_data.csv'
  EVAL_IMAGE_DATA = DATA_EXPLORING_ROOT / 'eval_image_data.csv'
  ANNOTATION_FILE = DATA_EXPLORING_ROOT / 'annotation.csv'
  CATEGORY_KEY_FILE = INPUT_ROOT / 'category_key.csv'
  SAMPLE_SUBMISSION_FILE = INPUT_ROOT / 'sample_submission.csv'

  # 기타 설정
  DATASET_CONFIG = OUTPUT_ROOT / 'dataset.yaml'
  MODEL_NAME = 'FathomNet-YOLOv8'

  N_EPOCHS = 50
  N_BATCH = 16
  RANDOM_STATE = 2023
  SAMPLE_SIZE = 1.0
  TEST_SIZE =.2
  INDEX = 'id'

- fastai :

  https://teamlab.github.io/jekyllDecent/blog/tutorials/fast.ai-%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC-%EC%84%A4%EC%B9%98%ED%95%98%EA%B3%A0-CNN-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# seaborn
custom_params = {
    'lines.linewidth': 1,
}

blues_palette = palette = sns.color_palette("Blues_r", n_colors=20)
reds_palette = palette = sns.color_palette("Reds_r", n_colors=20)
greys_palette = palette = sns.color_palette("Greys", n_colors=10)

blue = blues_palette[1]
red = reds_palette[1]
two_colors = [blue, red]

sns.set()
sns.set_theme(
    style = 'whitegrid',
    palette = blues_palette,
    rc = custom_params
)

### Data Loading

In [ ]:
def read_train_data(file=Cfg.TRAIN_IMAGE_DATA, index_col = Cfg.INDEX):
  return pd.read_csv(file).set_index(Cfg.INDEX)
